In [321]:
import numpy as np 
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,UpSampling2D,InputLayer,Reshape

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import matplotlib.pyplot as plt

from keras.layers import LeakyReLU
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from sklearn.utils import Bunch
import math
import pandas as pd
import datetime
import time
import csv
import numpy as np

from sklearn.model_selection import train_test_split
import cv2

import os
from PIL import Image
import sys


In [322]:
#from PIL import Image, ImageOps
white = [255,255,255]
def pad_img(x,scale):  
        #print(x.shape)
        h,w = x.shape[:2]
        h=min(h,scale[1])
        w=min(w,scale[0])
        diff_vert = scale[1] - h
        pad_top = diff_vert//2
        pad_bottom = diff_vert - pad_top
        diff_hori = scale[0] - w
        pad_left = diff_hori//2
        pad_right = diff_hori - pad_left
        x=cv2.copyMakeBorder(x, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=white)
        #assert x.shape[:2] == scale 
        #padding = (pad_left, pad_top, pad_right, pad_bottom)
        #x=ImageOps.expand(x, padding)
        #x =cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        #print(x.shape)
        #print("-----------------",x)
        return x;

In [323]:
def getData(pathd,scale):
    
    Alldatas=[]
    img_data=[]
    
    img_data.append(pathd)
        
    for i in img_data:
        img=load_img(i)
        x=img_to_array(img)
        h,w = x.shape[:2]
        print(h,w)
        if(w>scale[0] or h>scale[1]):
            hh=scale[1];
            ww=scale[0];
            if(h%scale[1]!=0):
                ff=h/scale[1];
                hh=((int(ff)+1)*scale[1])

            if(w%scale[0]!=0):
                ff=w/scale[0];
                ww=((int(ff)+1)*scale[0])

            new_scale=(ww,hh)
            x=pad_img(x,new_scale)
            
        else:
            x=pad_img(x,scale)
        Alldatas.append(x)
    return Alldatas

In [324]:
def prepare(arr):
    arr=np.asarray(arr,dtype="float32")
    arr2=arr/255
    return arr2

In [325]:
from keras.models import load_model
model = load_model(r"E:\class note\2020-Fall\thesis\model_pc3_1.h5")

In [326]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
from keras.preprocessing import image
from keras.models import load_model


top=tk.Tk()
top.geometry('800x600')
top.title('For Image Pre-Processing')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)

In [327]:
def classify(file_path):
        
    global label_packed
    scale=(540,420)
     
    all_img=getData(file_path,scale)
    
    all_img=tf.image.rgb_to_grayscale(all_img)
    test_t = prepare(all_img)
    
    for k in range(0,len(test_t)):
        orginal=all_img[k]
        output=test_t[k]

        height,width=output.shape[:2]

        if(width>scale[0] or height>scale[1]):

            imag = {}

            width=int(width/scale[0])
            height=int(height/scale[1])

            print(width,height)
            w=scale[0]
            h=scale[1]


            for i in range(0,height):
                for j in range(0,width):
                    ws=j*w;
                    we=(j+1)*w;
                    hs=i*h
                    he=(i+1)*h
                    crop_img1= output[hs:he,ws:we]

                    #print(crop_img1.shape)
                    #im=array_to_img(crop_img1)
                    #plt.imshow(im,cmap="gray")
                    #plt.show()
                    predicted_label=model.predict(crop_img1.reshape((1,)+crop_img1.shape)).reshape(scale[1],scale[0],1)
                    imag[(i,j)]=(predicted_label)

            ff=0
            for i in range(0,height):
                for j in range(0,width):
                    ff+=1
                    #print(imag[(i,j)]);
                    im=array_to_img(imag[(i,j)])
                    #print(j)
                    #plt.imshow(im,cmap="gray")
                    #plt.show()
                    cv2.imwrite(r'E:\class note\2020-Fall\pc2_2\\'+str(ff)+'.jpg',np.array(im))

            v1=imag[(0,0)]
            for j in range(1,width):
                v1 = np.concatenate((v1, imag[(0,j)]), axis=1)        
            v2=v1

            for i in range(1,height):
                vv=imag[(i,0)]
                for j in range(1,width):     
                    vv=np.concatenate((vv, imag[(i,j)]), axis=1)
                v2=np.concatenate((v2, vv), axis=0)


            #plt.figure(figsize=(50,50))
            #plt.subplot(10,10,1)
            #print(check)
            #xx=array_to_img(check)
            temp=k;
            #cv2.imwrite('input'+str(temp)+'.jpg',np.array(xx))
            #plt.imshow(array_to_img(orginal),cmap="gray")
            #plt.subplot(10,10,2)
            #print(predicted_label)
            xx=array_to_img(v2)
            cv2.imwrite(r'E:\class note\2020-Fall\pc3_1\\'+str(temp)+'.jpg',np.array(xx))
            #plt.imshow(xx,cmap="gray")


        else:

            predicted_label=model.predict(output.reshape((1,)+output.shape)).reshape(scale[1],scale[0],1)
            #plt.figure(figsize=(50,50))
            #plt.subplot(10,10,1)
            #print(check)
            #xx=array_to_img(check)
            temp=k;
            #cv2.imwrite('input'+str(temp)+'.jpg',np.array(xx))
            #plt.imshow(array_to_img(orginal),cmap="gray")
            #plt.subplot(10,10,2)
            #print(predicted_label)
            xx=array_to_img(predicted_label)
            cv2.imwrite(r'E:\class note\2020-Fall\pc2_1\\'+str(temp)+'.jpg',np.array(xx))
            #plt.imshow(xx,cmap="gray")
    
    top.destroy()    

In [328]:
def show_classify_button(file_path):
    classify_b=Button(top,text="Run",command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)
        

def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),
    (top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass

 
upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Image",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()
top.mainloop()

2206 1511
3 6
